In [1]:
import os
import matplotlib.pyplot as plt

In [8]:
dirs = os.listdir()
counter = 0
for d in dirs:
    if os.path.isdir(d) == True and 'ipynb' not in d:
        os.chdir(d)
        filenames = os.listdir()
        for f in filenames:
            counter += 1
            img = plt.imread(f)
            new_filename = f'../{counter:03d}_{d}.jpg'
            plt.imsave(new_filename, img)
        os.chdir('..')

In [5]:
counter = 1
f'{counter:03d}'

'001'

In [6]:
plt.imsave?

Signature:
plt.imsave(
    fname: 'str | os.PathLike | BinaryIO',
    arr: 'ArrayLike',
    **kwargs,
) -> 'None'
Docstring:
Colormap and save an array as an image file.

RGB(A) images are passed through.  Single channel images will be
colormapped according to *cmap* and *norm*.

.. note::

   If you want to save a single channel image as gray scale please use an
   image I/O library (such as pillow, tifffile, or imageio) directly.

Parameters
----------
fname : str or path-like or file-like
    A path or a file-like object to store the image in.
    If *format* is not set, then the output format is inferred from the
    extension of *fname*, if any, and from :rc:`savefig.format` otherwise.
    If *format* is set, it determines the output format.
arr : array-like
    The image data. The shape can be one of
    MxN (luminance), MxNx3 (RGB) or MxNx4 (RGBA).
vmin, vmax : float, optional
    *vmin* and *vmax* set the color scaling for the image by fixing the
    values that map to the colo